In [4]:
import MeCab
import unidic

tagger = MeCab.Tagger()  # 「tagger = MeCab.Tagger('-d ' + unidic.DICDIR)」
sample_txt = 'くるまでまつ'
result = tagger.parse(sample_txt)
print(result)

くる	動詞,非自立可能,,,カ行変格,連体形-一般,クル,来る,くる,クル,くる,クル,和,"","","","","","",用,クル,クル,クル,クル,"1","C1","",2891174448865985,10518
まで	助詞,副助詞,,,,,マデ,まで,まで,マデ,まで,マデ,和,"","","","","","",副助,マデ,マデ,マデ,マデ,"","名詞%F2@1,形容詞%F2@1,動詞%F2@1","",9865651581755904,35891
まつ	動詞,一般,,,五段-タ行,連体形-一般,マツ,待つ,まつ,マツ,まつ,マツ,和,"","","","","","",用,マツ,マツ,マツ,マツ,"1","C1","",9848884029432513,35830
EOS



In [5]:
wakati = MeCab.Tagger('-Owakati')
result = wakati.parse(sample_txt)
print(sample_txt)
print(result)

くるまでまつ
くる まで まつ 



In [36]:
text = """
マイクロソフト（英: Microsoft Corporation）は、アメリカ合衆国ワシントン州に本社を置く、ソフトウェアを開発、販売する会社である。2021年11月時点での時価総額が約270兆円を超え世界2位[3]。1975年にビル・ゲイツとポール・アレンによって創業された[注釈 1]。1985年にパソコン用OSのWindowsを開発。1990年にWindows向けのオフィスソフトとしてMicrosoft Officeを販売。1995年にウェブブラウザのInternet Explorerをリリース。2001年に家庭用ゲーム機のXboxを販売。2009年に検索エンジンのBingを設立。2010年にクラウドサービスとしてAzureを開始。
"""
result = wakati.parse(text)
print(result)

マイクロ ソフト （ 英 : Microsoft Corporation ） は 、 アメリカ 合衆 国 ワシントン 州 に 本社 を 置く 、 ソフトウェア を 開発 、 販売 する 会社 で ある 。 2021 年 11 月 時点 で の 時価 総額 が 約 270 兆 円 を 超え 世界 2 位 [ 3 ] 。 1975 年 に ビル ・ ゲイツ と ポール ・ アレン に よっ て 創業 さ れ た [ 注釈 1 ] 。 1985 年 に パソコン 用 OS の Windows を 開発 。 1990 年 に Windows 向け の オフィス ソフト と し て Microsoft Office を 販売 。 1995 年 に ウェブ ブラウザ の Internet Explorer を リリース 。 2001 年 に 家庭 用 ゲーム 機 の Xbox を 販売 。 2009 年 に 検索 エンジン の Bing を 設立 。 2010 年 に クラウド サービス と し て Azure を 開始 。 



In [37]:
import MeCab

m = MeCab.Tagger('') #mecabのtagger objectの宣言

# https://qiita.com/shimajiroxyz/items/e44058af8b036f5354aa
def bunsetsuWakachi(text):
    m_result = m.parse(text).splitlines()
    m_result = m_result[:-1] #最後の1行は不要な行なので除く
    break_pos = ['名詞','動詞','接頭詞','副詞','感動詞','形容詞','形容動詞','連体詞'] #文節の切れ目を検出するための品詞リスト
    wakachi = [''] #分かち書きのリスト
    afterPrepos = False #接頭詞の直後かどうかのフラグ
    afterSahenNoun = False #サ変接続名詞の直後かどうかのフラグ
    for v in m_result:
        if '\t' not in v: continue
        surface = v.split('\t')[0] #表層系
        pos = v.split('\t')[1].split(',') #品詞など
        pos_detail = ','.join(pos[1:4]) #品詞細分類（各要素の内部がさらに'/'で区切られていることがあるので、','でjoinして、inで判定する)
        #この単語が文節の切れ目とならないかどうかの判定
        noBreak = pos[0] not in break_pos
        noBreak = noBreak or '接尾' in pos_detail
        noBreak = noBreak or (pos[0]=='動詞' and 'サ変接続' in pos_detail)
        noBreak = noBreak or '非自立' in pos_detail #非自立な名詞、動詞を文節の切れ目としたい場合はこの行をコメントアウトする
        noBreak = noBreak or afterPrepos
        noBreak = noBreak or (afterSahenNoun and pos[0]=='動詞' and pos[4]=='サ変・スル')
        if noBreak == False:
            wakachi.append("")
        wakachi[-1] += surface
        afterPrepos = pos[0]=='接頭詞'
        afterSahenNoun = 'サ変接続' in pos_detail
    if wakachi[0] == '': wakachi = wakachi[1:] #最初が空文字のとき削除する
    return wakachi

text_list = bunsetsuWakachi(text)

In [41]:
hoge_list = []
MAX_LEN = 7

tmp_str = ""
for text in text_list:

    # 指定文字数よりも小さければ追加
    if len(tmp_str) + len(text)<MAX_LEN:
        tmp_str+=text
    # 指定文字数よりも大きければ表示、リセット
    else:
        hoge_list.append(tmp_str)
        print(tmp_str)
        tmp_str=text
hoge_list.append(tmp_str)

マイクロ
ソフト（英:
Microsoft
Corporation）は、
アメリカ
合衆国
ワシントン
州に
本社を置く、
ソフトウェアを
開発、
販売する
会社である。
2021年
11月
時点での時価
総額が約
270兆円を
超え世界
2位[3]。
1975年に
ビル・
ゲイツと
ポール・
アレンに
よって
創業された[
注釈1]。
1985年に
パソコン用
OSの
Windowsを
開発。
1990年に
Windows向けの
オフィス
ソフトとして
Microsoft
Officeを
販売。
1995年に
ウェブ
ブラウザの
Internet
Explorerを
リリース。
2001年に
家庭用ゲーム
機の
Xboxを
販売。
2009年に
検索
エンジンの
Bingを
設立。
2010年に
クラウド
サービスとして
Azureを


In [33]:
import PIL.Image
import PIL.ImageDraw
import PIL.ImageFont

# 使うフォント，サイズ，描くテキストの設定
#ttfontname = "data/font/ipaexg.ttf" # ゴシック体
ttfontname = "data/font/ipaexg.ttf" # 明朝体
fontsize = 36
text = "bbbbbbbbbbb"

# 画像サイズ，背景色，フォントの色を設定
canvasSize    = (600, 300)
backgroundRGB = (255, 255, 255)
textRGB       = (0, 0, 0)

# 文字を描く画像の作成
img  = PIL.Image.new('RGB', canvasSize, backgroundRGB)
draw = PIL.ImageDraw.Draw(img)

# 用意した画像に文字列を描く
font = PIL.ImageFont.truetype(ttfontname, fontsize)
textWidth, textHeight = draw.textsize(text,font=font)
textTopLeft = (canvasSize[0]//2-textWidth//2, canvasSize[1]//2-textHeight//2) # 前から1/6，上下中央に配置
draw.text(textTopLeft, text, fill=textRGB, font=font)

img.save("data/tmp_img/image3.png")

In [45]:
def make_image(text,index):

    # 使うフォント，サイズ，描くテキストの設定
    #ttfontname = "data/font/ipaexg.ttf" # ゴシック体
    ttfontname = "data/font/ipaexg.ttf" # 明朝体
    fontsize = 36


    # 画像サイズ，背景色，フォントの色を設定
    canvasSize    = (600, 300)
    backgroundRGB = (255, 255, 255)
    textRGB       = (0, 0, 0)

    # 文字を描く画像の作成
    img  = PIL.Image.new('RGB', canvasSize, backgroundRGB)
    draw = PIL.ImageDraw.Draw(img)

    # 用意した画像に文字列を描く
    font = PIL.ImageFont.truetype(ttfontname, fontsize)
    textWidth, textHeight = draw.textsize(text,font=font)
    textTopLeft = (canvasSize[0]//2-textWidth//2, canvasSize[1]//2-textHeight//2) # 前から1/6，上下中央に配置
    draw.text(textTopLeft, text, fill=textRGB, font=font)

    img.save(f"data/tmp_img/{str(index).zfill(4)}.png")


for index,text in enumerate(hoge_list):
    make_image(text,index)

In [43]:
import cv2
import glob

size=(600,300)#サイズ指定
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')#保存形式
save = cv2.VideoWriter('next_tegaki_0mv.mp4',fourcc,4,size)#動画を保存する形を作成

print("保存中...")

#0フォルダ
pic_data=glob.glob("data/tmp_img/*.png")#0フォルダだけ取り出す
for i in range(len(pic_data)):#178回繰り返す
    img=pic_data[i]
    img=cv2.imread(img)#画像を読み込む
    img=cv2.resize(img,(600,300))#上でサイズを指定していますが、念のため
    save.write(img)#保存

print("保存しました")

save.release() # ファイルを閉じる

保存中...
保存しました


In [46]:
import os
import shutil

shutil.rmtree("data/tmp_img/")
os.mkdir("data/tmp_img/")

PermissionError: [WinError 5] アクセスが拒否されました。: 'data/tmp_img/'